In [ ]:
#測試抓愛食記網站
import requests
from bs4 import BeautifulSoup
from datetime import date,datetime
import re
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import csv
import pandas as pd
import json
import random

In [ ]:
def get_comment(url, comment_ary):
    browser = webdriver.Chrome(executable_path='/usr/bin/chromedriver')
    browser.get(url)
    browser.implicitly_wait(1)
    sval = random.randint(1,3)
    time.sleep(sval)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    while len(soup.select('.user-checkin > .content > .btn-more-checkin-wrapper')) > 0:
        browser.find_element_by_xpath("//button[@class='jss76 jss50 jss52 jss55 btn-more-checkin']").click()
        sval = random.randint(1,3)
        time.sleep(sval)
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        
    user_num = {}   
    for i in range(len(soup.select('.checkin-list-outer > .comment-block-outer'))):  
        comment_member = {}
        comment_restaurant = soup.select('.restaurant-name-outer > h1 > a')[0].text
        #print(comment_restaurant)
        comment_member['restaurant'] = comment_restaurant
        
        
        user_num = soup.select('.checkin-list-outer > .comment-block-outer > .comment-block > .comment-content-outer')[i]
        account = user_num.select('a')[0]['href']
        account = account[-24:]
        #print(account)
        comment_member['account'] = account 
        
        try:
            name_path = '.content > .checkin-list-outer > div:nth-child(' + str(i+1) +') > .comment-block > .comment-content-outer > div > .username-outer > .username'
            name = soup.select(name_path)[0].text
            #print(name)    
        except IndexError:
            print("name wrong")
            name = "鄉民"
            pass
        comment_member['name'] = name
    
        try:
            level_path = '.content > .checkin-list-outer > div:nth-child(' + str(i+1) + ') > .comment-block > .comment-content-outer > div > .username-outer > a:nth-child(2) > span'
            level = soup.select(level_path)[0].text
            #print(level)
        except IndexError:
            print("comment level wrong")
            level = "普通會員"
            pass
        comment_member['level'] = level
        
        
        try:      
            comment_star = 0
            comment_star_path = '.content > .checkin-list-outer > div:nth-child(' + str(i+1) + ') > .comment-block > .comment-content-outer > div > .info-outer > div > div > .text'        
            comment_star = soup.select( comment_star_path)[0].text
            #print(comment_star)
        except IndexError:   
            comment_star = 0
            print("comment star wrong")
            pass
        comment_member['comment_star'] =  comment_star
        
        
        
        comment_path = '.content > .checkin-list-outer > div:nth-child(' + str(i+1) + ') > .comment-block > .comment-content-outer > div > .message'
        comment = soup.select(comment_path)[0].text
        #print(comment)
        comment_member['comment'] = comment 
        #print(i)
        #print(comment_member)
        #print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
        #ave data to array
        comment_ary.append(comment_member)  
        #print(comment_ary)
        #print('=============================================================================================')

#     for j in comment_ary:
#         print(j)
#         print('=============================================================================================')
    #print(comment_ary)
    sval = random.randint(4,6)
    time.sleep(sval)
    browser.close() 

In [ ]:
def pageDetail(detail_url, restaurant_series_num, comment_ary):
    #print("url:")
    #print( detail_url)
    save_data = {}
    res = requests.get(detail_url)
    soup =  BeautifulSoup(res.text)
    
    #test tool
    #tag = input("請輸入定位元素，class前面加上.，id前面加上# ")  
    #for drink in soup.select('{}'.format(tag)):
        #print(drink.get_text())
    save_data['restaurant_series_num'] = restaurant_series_num
    
    
    try:
        name =  soup.select('.title-outer > .title')[0].text
        #print(name)
    except IndexError: 
        name = "沒店名"
        print("no store name")
        pass
    save_data['name'] = name    
    
    try:
        star = soup.select('.jsx-1207467136')[0].text
        #print(star)    
    except IndexError:
        star = 0
        print("no star")
        pass
    save_data['star'] = star
    
    
    
    try:
        comment_num = soup.select('.count')[0].text
        comment_num = int(comment_num[:-3])
        #print(comment_num)  
    except IndexError:
        comment_num = 0
        print("comment_num wrong")
        pass
    save_data['comment_num'] = comment_num
    
    
    data = soup.select('.info-outer > .info > div > span')
    
    try:
        address_check = data[0].text
        #print(address_check)
        address = ""
        if address_check == "店家地址|":
            address = data[1].text
            #print(address)
    except IndexError:
        address = "null"
        print("address wrong")
        pass
    
    save_data['address'] = address
    
    
    
    try:
        phone_check = data[2].text
        #print(phone_check)
        if phone_check == "聯絡電話|":
            phone = data[3].text
            #print(phone)   
        else:
            phone = 'null'
    except IndexError:
        phone = 'null'
        print("phone wrong")
        pass   
    save_data['phone'] = phone
    
    
    try:      
        avg_price_check = data[4].text
        #print(avg_price_check)
        if avg_price_check == "均消價位|":
            avg_price = data[5].text
            avg_price = avg_price[4:]
        else:
            avg_price = 0   
        #print(avg_price)
    except IndexError:   
        avg_price = 0
        print("avg price wrong")
        pass
    save_data['avg_price'] = avg_price    

    get_comment(detail_url, comment_ary)

    print(save_data)
    print('-------------------------')
    return save_data

    
    

In [ ]:
def deal_with_restaurant_series_num(num):
    RSN = 'RSN_0'
    length = len(str(num))
    if length == 1:
        RSN = RSN + '0000' + str(num)
    elif length == 2:
        RSN = RSN + '000' + str(num)
    elif length == 3:
        RSN = RSN + '00' + str(num) 
    elif length == 4:
        RSN = RSN + '0' + str(num)  
    elif length == 5:
        RSN = RSN + str(num)      
    #print(RSN)
    return RSN

In [ ]:
#for test
test_RSN = deal_with_restaurant_series_num(657)
print(test_RSN)

In [ ]:
domain = 'https://ifoodie.tw'
restaurant_ary = []
comment_ary = []
counter = 0
restaurant_series_num_counter = 0
for page in range(1,5):
    restaurant_url = 'https://ifoodie.tw/explore/%E5%8F%B0%E5%8C%97%E5%B8%82/list?page={}'
    section = requests.get(restaurant_url.format(page))
    soup = BeautifulSoup(section.text)
    counter = 0   
    restaurant_data = {}
    restaurant_type = {}
    for restaurant in soup.select('.restaurant-info'):      
        catergory = soup.select('.category-row')[counter].text
        catergory = catergory[4:]
        #print(catergory)
        if catergory != '':
            restaurant_type['type'] = catergory
        else:
            restaurant_type['type'] = '小吃'
            
        #print(restaurant_type)
        dic = {}
        dic['link'] = domain + restaurant.select('a')[0]['href']
        restaurant_series_num = deal_with_restaurant_series_num(restaurant_series_num_counter)
        restaurant_data = pageDetail(dic['link'] , restaurant_series_num, comment_ary)
        restaurant_data.update(restaurant_type)
        restaurant_ary.append(restaurant_data)
        counter = counter + 1
        restaurant_series_num_counter = restaurant_series_num_counter + 1

In [ ]:
#ex comment member number
str = '59b901a222613946771733d2'
print(len(str))

In [ ]:
for j in restaurant_ary:
    print(j)
    print('------------------------------------------------------------------------------------------------------------------------------------------------------------')

In [ ]:
sub_restaurant = pd.DataFrame(restaurant_ary)

In [ ]:
sub_restaurant

In [ ]:
sub_restaurant.to_csv("raw_data/restaurant.csv", index=False)
sub_restaurant.to_json(r'raw_data/restaurant.json')

In [ ]:
for j in comment_ary:
    print(j)
    print('------------------------------------------------------------------------------------------------------------------------------------------------------------')

In [ ]:
sub_comment = pd.DataFrame(comment_ary)

In [ ]:
sub_comment

In [ ]:
sub_comment.to_csv("raw_data/comment.csv", index=False)
sub_comment.to_json(r'raw_data/comment.json')